# Data Preprocessing 

In this notebook we clean and prepare the data for analysis.

In [48]:
import os
from dateutil.parser import parse 
import numpy as np
import pandas as pd
from itertools import chain

# Covid Data 

We sourced the data from the John Hopkins Covid Github page, where they already aggregate and clean multiple sources.

In [49]:
covid_dir = os.path.join(os.pardir, 'data', 'raw', 'covid19', 'daily_reports')
daily_repports = os.listdir(covid_dir)

In [58]:

dt_obj = parse(daily_repports[0].split('.')[0])

pt_to_file = os.path.join(covid_dir, daily_repports[0])
pdf = pd.read_csv(pt_to_file)
pdf_2 = pdf[['Country_Region', 'Confirmed', 'Deaths', 'Recovered', 'Active', 'Incident_Rate', 'Case_Fatality_Ratio']]
pdf_3 = pdf_2.groupby(['Country_Region']).aggregate({'Confirmed':'sum', 'Deaths':'sum', 'Recovered':'sum','Active':'mean','Incident_Rate':'mean','Case_Fatality_Ratio':'mean'}).reset_index()
pdf_3['year'] = dt_obj.year
pdf_3

,Country_Region,Confirmed,Deaths,Recovered,Active,Incident_Rate,Case_Fatality_Ratio,year
0,Afghanistan,52513,2201,41727,8585.0,134.896578,4.191343,2021
1,Albania,58316,1181,33634,23501.0,2026.409062,2.025173,2021
2,Algeria,99897,2762,67395,29740.0,227.809861,2.764848,2021
3,Andorra,8117,84,7463,570.0,10505.403482,1.034865,2021
4,Angola,17568,405,11146,6017.0,53.452981,2.305328,2021
...,...,...,...,...,...,...,...,...
190,Vietnam,1474,35,1325,114.0,1.514302,2.374491,2021
191,West Bank and Gaza,139223,1418,118926,18879.0,2729.105017,1.018510,2021
192,Yemen,2101,610,1396,95.0,7.044197,29.033793,2021
193,Zambia,20997,390,18773,1834.0,114.213720,1.857408,2021


In [59]:
daily_repports = os.listdir(covid_dir)
dt_obj = parse(daily_repports[-1].split('.')[0])

pt_to_file = os.path.join(covid_dir, daily_repports[-1])
pdf = pd.read_csv(pt_to_file)
pdf_2 = pdf[['Country_Region', 'Confirmed', 'Deaths', 'Recovered', 'Active', 'Incident_Rate', 'Case_Fatality_Ratio']]
pdf_4 = pdf_2.groupby(['Country_Region']).aggregate({'Confirmed':'sum', 'Deaths':'sum', 'Recovered':'sum','Active':'mean','Incident_Rate':'mean','Case_Fatality_Ratio':'mean'}).reset_index()
pdf_4['year'] = dt_obj.year
pdf_4

,Country_Region,Confirmed,Deaths,Recovered,Active,Incident_Rate,Case_Fatality_Ratio,year
0,Afghanistan,52330,2189,41727,8414.0,134.426484,4.183069,2020
1,Albania,58316,1181,33634,23501.0,2026.409062,2.025173,2020
2,Algeria,99610,2756,67127,29727.0,227.155372,2.766790,2020
3,Andorra,8049,84,7432,533.0,10417.394681,1.043608,2020
4,Angola,17553,405,11044,6104.0,53.407342,2.307298,2020
...,...,...,...,...,...,...,...,...
190,Vietnam,1465,35,1325,105.0,1.505056,2.389078,2020
191,West Bank and Gaza,138004,1400,117183,19421.0,2705.209691,1.014463,2020
192,Yemen,2099,610,1394,95.0,7.037492,29.061458,2020
193,Zambia,20725,388,18660,1677.0,112.734169,1.872135,2020


In [52]:
pdf_cat =pd.concat([pdf_3, pdf_4])
pdf_cat[pdf_cat['Country_Region'] == 'Afghanistan']

,Country_Region,Confirmed,Deaths,Recovered,Active,Incident_Rate,Case_Fatality_Ratio,year
0,Afghanistan,52513,2201,41727,8585.0,134.896578,4.191343,2021
0,Afghanistan,52330,2189,41727,8414.0,134.426484,4.183069,2020


In [53]:
pdf_cat.columns

Index(['Country_Region', 'Confirmed', 'Deaths', 'Recovered', 'Active',
       'Incident_Rate', 'Case_Fatality_Ratio', 'year'],
      dtype='object')

In [57]:
pdf_cat2= pdf_cat.groupby(['Country_Region', 'year']).aggregate({'Confirmed':'max', 'Deaths':'max', 'Recovered':'max','Active':'mean','Incident_Rate':'mean','Case_Fatality_Ratio':'mean'}).reset_index()
pdf_cat2[pdf_cat2['Country_Region'] == 'Afghanistan']

,Country_Region,year,Confirmed,Deaths,Recovered,Active,Incident_Rate,Case_Fatality_Ratio
0,Afghanistan,2020,52330,2189,41727,8414.0,134.426484,4.183069
1,Afghanistan,2021,52513,2201,41727,8585.0,134.896578,4.191343
